In [58]:
from heston import UnivariateHestonSV
UnivariateHestonSV.heston_call_px?

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.optimize import least_squares
%matplotlib inline

from heston import UnivariateHestonSV, MultivariateHestonSV
from heston import make_surface
from heston import MultiAssetsOption, MultiAssetsAsianPut, MultiAssetsAsianCall
from heston import MultiAssetsDiscreteKIEuropeanPut, MultiAssetsAsianZeroCostCollar
from heston import MultiAssetsBestOfAsianCall, MultiAssetsWorstOfDiscreteKIEuropeanPut
from heston import structure_constructor, structure_constructor_best_eu

In [2]:
data = pd.read_excel('data/equities.xlsx', index_col=0)
assets = data.corp.unique()
data.head()

,corp,strike,mat,iv,S0,adv,tot_shr,owned_shr,avg_buy_px,bs_px
0,Green Co.,30,0.083333,NaN,34,217000,30,4.25,31.5,NaN
1,Green Co.,30,0.250000,0.3747,34,217000,30,4.25,31.5,5.143945
2,Green Co.,30,0.500000,0.3686,34,217000,30,4.25,31.5,6.126322
3,Green Co.,35,0.083333,0.4211,34,217000,30,4.25,31.5,1.270197
4,Green Co.,35,0.250000,0.3418,34,217000,30,4.25,31.5,2.044304


In [3]:
assets = data.corp.unique()
mv_hes = MultivariateHestonSV(assets)
mv_hes.set_params(
    {
        # parameters list: ['kappa', 'v0', 'theta', 'eta', 'rho_sv']
        assets[0]: [12.22153077,  0.19999999,  0.10679624,  1.11942453, -0.79999979],
        assets[1]: [ 0.69115605,  0.11214887,  0.4196924 ,  0.19383575, -0.79890507],
        assets[2]: [ 2.0236936 ,  0.14899818,  0.30447942,  0.4452153 , -0.79999991],
        # risk free rate
        'r': 0.045,
        # stock covariance matrix
        'cov_s': np.array([
            [1, 0.3, 0.3],
            [0.3, 1, 0.3],
            [0.3, 0.3, 1]
        ])
    }
)

## Optimization

In [46]:
s_0 = np.array([34, 15, 25]) # time-0 stock price
# use pre-computed paths to speed things up
n = 1000
paths = mv_hes.simulate_paths_cfb(
    n_paths=n, n_nodes=252, T=2, dS=1e-4, S0=s_0)

100% (1000 of 1000) |#####################| Elapsed Time: 0:00:40 Time: 0:00:40


In [38]:
s_0 = np.array([34, 15, 25])
adv = data.adv.unique() / 1e6
markup = (np.array([0.1, 0.1, 0.1]), 0.0)

def metric(K, D, x, R, p):
    loss, structure, pct_adv, cap, deltas, slip = structure_constructor(
        model=mv_hes, x=x, put_Ks=K, Ds=D, R_req=R, adv=adv, 
        adv_thres=0.3, S0=s_0, cap_required=100, alpha=0.1, pre_computed_paths=paths, 
        premiums=p)
    delta_collar, delta_boc, delta_KI = deltas
    delta_tot = delta_collar-delta_boc+delta_KI
    # print(structure)
    print('Put Strikes:', K)
    print('Shares:', x)
    print('Loss:', loss)
    print('|X/ADV|:', np.abs(pct_adv))
    print('Price Slippage', slip)
    print('Pct Price Slippage (BPs)', (slip/s_0)*1e4)
    print('Dollar Slippage', slip.dot(delta_tot.reshape(3,))[0])
    print('Dollar Premium', -markup[0].dot(x)+markup[1])
    print('Locked-in Capital:', cap)
    print('Deltas/Sell x Collar:', delta_collar)
    print('Deltas/Sell 1 Best of Call:', -delta_boc)
    print('Deltas/Buy 1 KI:', delta_KI)
    print('Deltas/Entire Structure:', delta_tot)
    return structure

In [18]:
Kput = np.array([31.5, 10.05, 22])
D = np.array([2, 1.5, 1.8])
x = np.array([-0.4, -5.7, -1.83])
R = 0.05
p = markup = (np.array([0.0, 0.0, 0.0]), 0.8)


structure = metric(Kput, D, x, R, p)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [31.5  10.05 22.  ]
Shares: [-0.4  -5.7  -1.83]
Loss: 0.5258624523932963
|X/ADV|: [[0.61637967 0.58181681 0.61147152]]
Price Slippage [[1.28289008 0.40000316 0.80785036]]
Pct Price Slippage (BPs) [[377.32061121 266.66877014 323.14014377]]
Dollar Slippage 0.7097265454397614
Dollar Premium 0.8
Locked-in Capital: 100.66495040169944
Deltas/Sell x Collar: [[0.25042389 0.92944768 1.11400897]]
Deltas/Sell 1 Best of Call: [[-0.07966655 -0.45089035 -0.48094497]]
Deltas/Buy 1 KI: [[-0.03700295 -0.12364908 -0.14266383]]
Deltas/Entire Structure: [[0.13375439 0.35490825 0.49040016]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[31.500, 10.050, 22.000], T=2
- MultiAssetsAsianCall: K=[40.187, 26.157, 31.566], T=2
MultiAssetsBestOfAsianCall: K=[40.187, 26.157, 31.566], R_required=0.05, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[30.530, 6.670, 17.738], H=[28.530, 5.170, 15.938], T_entry=1, T_mature=2


In [37]:
s_0 = np.array([34, 15, 25])
adv = data.adv.unique() / 1e6
markup = (np.array([0.1, 0.1, 0.1]), 0.0)

def metric_eu(K, D, x, R, p):
    loss, structure, pct_adv, cap, deltas, slip = structure_constructor_best_eu(
        model=mv_hes, x=x, put_Ks=K, Ds=D, R_req=R, adv=adv, 
        adv_thres=0.3, S0=s_0, cap_required=100, alpha=0.1, pre_computed_paths=paths, 
        premiums=p)
    delta_collar, delta_boc, delta_KI = deltas
    delta_tot = delta_collar-delta_boc+delta_KI
    # print(structure)
    print('Put Strikes:', K)
    print('Shares:', x)
    print('Loss:', loss)
    print('|X/ADV|:', np.abs(pct_adv))
    print('Price Slippage', slip)
    print('Pct Price Slippage (BPs)', (slip/s_0)*1e4)
    print('Dollar Slippage', slip.dot(delta_tot.reshape(3,))[0])
    print('Dollar Premium', -markup[0].dot(x)+markup[1])
    print('Locked-in Capital:', cap)
    print('Deltas/Sell x Collar:', delta_collar)
    print('Deltas/Sell 1 Best of Call:', -delta_boc)
    print('Deltas/Buy 1 KI:', delta_KI)
    print('Deltas/Entire Structure:', delta_tot)
    return structure

In [39]:
Kput = np.array([30.5, 12.5, 21.5])
D = np.array([2, 1.5, 1.8])
x = np.array([-0.25, -3.7, -2.83])
R = 0.05
p = markup = (np.array([0.0, 0.0, 0.0]), 0.8)


structure = metric(Kput, D, x, R, p)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [30.5 12.5 21.5]
Shares: [-0.25 -3.7  -2.83]
Loss: 1.1328659296490973
|X/ADV|: [[0.32239506 1.28690232 0.85579446]]
Price Slippage [[0.34131967 0.74854352 1.07914265]]
Pct Price Slippage (BPs) [[100.38813811 499.0290129  431.65705877]]
Dollar Slippage 1.352159578558818
Dollar Premium 0.8
Locked-in Capital: 104.8461855743153
Deltas/Sell x Collar: [[0.13780072 1.7012646  1.60782766]]
Deltas/Sell 1 Best of Call: [[-0.04540168 -0.70526729 -0.71267078]]
Deltas/Buy 1 KI: [[-0.02243931 -0.21098689 -0.20880972]]
Deltas/Entire Structure: [[0.06995973 0.78501042 0.68634716]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[30.500, 12.500, 21.500], T=2
- MultiAssetsAsianCall: K=[41.459, 20.673, 32.345], T=2
MultiAssetsBestOfAsianCall: K=[41.459, 20.673, 32.345], R_required=0.05, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[30.116, 9.750, 17.150], H=[28.116, 8.250, 15.350], T_entry=1, T_mature=2


In [47]:
Kput = np.array([30.5, 12.5, 21.3])
D = np.array([2, 1.5, 1.8])
x = np.array([-0.25, -3.7, -2.8])
R = 0.05
p = markup = (np.array([0.0, 0.0, 0.0]), 0.8)


#structure = metric_eu(Kput, D, x, R, p)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[30.500, 12.500, 21.300], T=2
- MultiAssetsAsianCall: K=[41.459, 20.673, 32.661], T=2
MultiAssetsBestOfEuropeanCall: K=[41.459, 20.673, 32.661], R_required=0.05, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[30.963, 15.764, 26.279], H=[28.963, 14.264, 24.479], T_entry=1, T_mature=2


In [ ]:
Kput = np.array([30.5, 12.5, 21.3])
D = np.array([2, 1.5, 1.8])
x = np.array([-0.25, -3.7, -2.8])
R = 0.05
p = markup = (np.array([0.0, 0.0, 0.0]), 0.8)


structure = metric_eu(Kput, D, x, R, p)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

In [65]:
np.linspace(34*0.9, 34*1.1, 11)

array([30.6 , 31.28, 31.96, 32.64, 33.32, 34.  , 34.68, 35.36, 36.04,
       36.72, 37.4 ])

In [48]:
SSc, SSf, SSb, XXc, XXf, XXb, VV = paths
paths_c = (SSc, XXc, VV)
direction = [-1, -1, 1]
print('------------------------------------------')
for i, (k, obj) in enumerate(structure.items()):
    print(repr(obj))
    px, se = obj.mc_price(spots=s_0, pre_computed_paths=paths_c)
    delta = obj.mc_delta(spots=s_0, dS=1e-4, pre_computed_paths=paths)
    print('Price:', px*direction[i])
    print('(SE)', se)
    print('Delta', delta*direction[i])
    print('------------------------------------------')

------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[30.500, 12.500, 21.300], T=2
- MultiAssetsAsianCall: K=[41.459, 20.673, 32.661], T=2
Price: [ 0.00082412 -0.02946164  0.10251764]
(SE) [0.18338043 0.09680232 0.20181725]
Delta [[ 0.53648757 -0.         -0.        ]
 [-0.          0.46165104 -0.        ]
 [-0.         -0.          0.58946423]]
------------------------------------------
MultiAssetsBestOfEuropeanCall: K=[41.459, 20.673, 32.661], R_required=0.05, T_entry=1, T_mature=2
Price: -17.166322997434683
(SE) 1.2350381284434693
Delta [[-0.05664869 -1.13157508 -0.91908572]]
------------------------------------------
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[30.963, 15.764, 26.279], H=[28.963, 14.264, 24.479], T_entry=1, T_mature=2
Price: 18.39405681745416
(SE) 0.6772402658269178
Delta [[-0.01851877 -0.44895457 -0.38774279]]
------------------------------------------


In [50]:
d1 = np.array([0.53648757,0.46165104,0.58946423])
d2 = np.array([-0.05664869, -1.13157508, -0.91908572])
d3 = np.array([-0.01851877, -0.44895457, -0.38774279])
x = np.array([-0.25, -3.7, -2.8])
(x*d1 - d2 -d3) / adv

array([-0.27167941, -0.20914623, -0.42851787])

In [62]:
dW = np.random.multivariate_normal(np.array(
            [0] * (2 * 3)), mv_hes.cov, (52,1000))

In [63]:
dW.shape

(52, 1000, 6)